In [43]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from astropy.io import ascii
from utils import *
from scipy.stats import skew, kurtosis
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
datasets = "../datasets/SuperCOSMOS/"
uki823_df =ascii.read(datasets + "UKI823/sssedrpair.dat", guess=False, Reader=ascii.FastNoHeader).to_pandas()

In [ ]:
uki823_df.columns = col_names
#ukr823_df.columns = col_names
#ukj823_df.columns = col_names
uki823_df.head()

In [ ]:
uki823_df.describe()

In [5]:
#Normalise SDSS class labels. Form confusion matrix
normalise_sdss_class(uki823_df)
uki823_df[['CLASS', 'CLASS_SDSS']]
confusion_matrix(uki823_df['CLASS'], uki823_df['CLASS_SDSS'])

In [6]:
#Create first raw dataset
data_x_raw = uki823_df.iloc[:,:-1]
data_y_raw =uki823_df['CLASS_SDSS']

In [7]:
#Split raw dataset into raw train,val,test sets
from sklearn.model_selection import train_test_split
random_state = 1
X_train_raw,X_test_raw,y_train,y_test = train_test_split(data_x_raw,data_y_raw,test_size=0.1,random_state=random_state)
X_train_raw,X_val_raw,y_train,y_val = train_test_split(X_train_raw,y_train,test_size=2./9,random_state=random_state)

In [8]:
X_train_raw = X_train_raw.reset_index(drop=True)
X_val_raw = X_val_raw.reset_index(drop=True)
X_test_raw = X_test_raw.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

X_train_raw.head()

In [9]:
#Define classification function
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,roc_auc_score

def Classify_Function(x_train,y_train,x_val,y_val):
    names = ["Logistic Regression", #"Linear SVM", "RBF SVM",
             "Decision Tree", "Random Forest", "Neural Net (Multi-layer perceptron)"]

    classifiers = [
        LogisticRegression(),
        SVC(kernel="linear", probability=True, random_state=random_state),
        #SVC(kernel='rbf', probability=True, random_state=random_state),
        DecisionTreeClassifier(max_depth=10),
        RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state),
        MLPClassifier(max_iter=1000, random_state=random_state)]

    ca_score = {} # Classification accuracy
    F1_scores = {} #F1 scores
    tr_score = {}
    tr_F1 = {}

    for name, clf in zip(names, classifiers):
        clf.fit(x_train, y_train)
        tr_score[name] = clf.score(x_train, y_train)
        ca_score[name] = clf.score(x_val, y_val)
        tr_F1[name] = f1_score(y_train, clf.predict(x_train), average='macro')
        F1_scores[name] = f1_score(y_val,clf.predict(x_val),average='macro')

    print('Classification performance on validation set:')
    for clf in names:
        print ("{}, training accuracy: {:.3f}, training f1: {:.3f}".format(clf, tr_score[clf], tr_F1[clf]))
        print ("{}, accuracy: {:.3f}, f1-score: {:.3f}\n\n".format(clf, ca_score[clf], F1_scores[clf]))


In [ ]:
# %%time
# #Classify the raw data
# Classify_Function(X_train_raw,y_train,X_val_raw,y_val)

In [ ]:
# labels = np.array([1,2])
# fig, ax = plt.subplots((len(X_train_raw.columns)), labels.size, figsize=(15,45), sharey = 'row', sharex = 'row')


# for ii, feat in enumerate(X_train_raw):
#     for jj, clas in enumerate(labels):
#         sns.distplot(X_train_raw[y_train==clas][feat], ax=ax[ii][jj], kde=True)
#         ax[ii][jj].xaxis.label.set_visible(False)
           
# [ax[0][ii].set_title("Class {}".format(clas)) for ii, clas in enumerate(labels)]
# [ax[ii][0].set_ylabel("{}".format(feat)) for ii, feat in enumerate(X_train_raw)]
        
# fig.tight_layout()
# plt.show()


In [10]:
X_train = X_train_raw.iloc[:,relevant_indices[0:(len(relevant_indices)-1)]]
X_val = X_val_raw.iloc[:,relevant_indices[0:(len(relevant_indices)-1)]]
X_test = X_test_raw.iloc[:,relevant_indices[0:(len(relevant_indices)-1)]]

In [11]:
#Extracting relevant columns and adding ellips and FF to X datasets
X_train = add_filling_factor(add_ellipticity_df(X_train))
X_val = add_filling_factor(add_ellipticity_df(X_val))
X_test = add_filling_factor(add_ellipticity_df(X_test))
X_train.columns

Index(['AREA', 'IPEAK', 'COSMAG', 'ISKY', 'A_U', 'B_U', 'THETA_U', 'A_I',
       'B_I', 'THETA_I', 'BLEND', 'QUALITY', 'PRFMAG', 'C_COSMAG', 'C_PRFMAG',
       'RA_SDSS', 'DEC_SDSS', 'GMAG_SDSS', 'RMAG_SDSS', 'IMAG_SDSS',
       'ELLIPTICITY', 'FILLING_FACTOR'],
      dtype='object')

In [ ]:
# #Identify features that look interesting...
# interesting_cols = [0, 1, 2, 10, 11, 21]

# # Need to add X_train and y_train back together here
# temp = deepcopy(X_train)
# temp['ys'] = y_train

# sns.pairplot(temp, vars=X_train.columns[interesting_cols], hue='ys', diag_kind = 'kde', plot_kws={'s' : 6})
# plt.show()


In [12]:
#Scale dataset and classify
from sklearn.preprocessing import StandardScaler
sc = StandardScaler().fit(X_train.astype('float64'))


# Classify_Function(sc.transform(X_train.astype('float64')),
#                   y_train,
#                   sc.transform(X_val.astype('float64')),
#                   y_val)

In [ ]:
# #Fitting PCA to scaled training data. 9 PC's
# from sklearn.decomposition import PCA
# pca = PCA().fit(sc.transform(X_train.astype('float64')))
# print(np.cumsum(pca.explained_variance_ratio_))


In [ ]:
# #Calculate PCA scores for all datasets. 11 PCs
# eleven_pcs = PCA(n_components = 11).fit(sc.transform(X_train.astype('float64')))
# pc_scores_train = eleven_pcs.transform(sc.transform(X_train.astype('float64')))
# pc_scores_val = eleven_pcs.transform(sc.transform(X_val.astype('float64')))
# pc_scores_test = eleven_pcs.transform(sc.transform(X_test.astype('float64')))

In [ ]:
# #Define function to plot PC directions
# def scatter_2d_label(X_2d, y, s=2, alpha=0.5, lw=2):
#     """Visualuse a 2D embedding with corresponding labels.
    
#     X_2d : ndarray, shape (n_samples,2)
#         Low-dimensional feature representation.
    
#     y : ndarray, shape (n_samples,)
#         Labels corresponding to the entries in X_2d.
        
#     s : float
#         Marker size for scatter plot.
    
#     alpha : float
#         Transparency for scatter plot.
        
#     lw : float
#         Linewidth for scatter plot.
#     """
#     targets = np.unique(y)
#     colors = sns.color_palette(n_colors=targets.size)
#     for color, target in zip(colors, targets):
#         plt.scatter(X_2d[y == target, 0], X_2d[y == target, 1], color=color, label=target, s=s, alpha=alpha, lw=lw)


In [ ]:
# dim_1 = 0# First dimension
# dim_2 = 1 # Second dimension
# plt.figure(figsize=(8,5)) # Initialise a figure instance with defined size
# scatter_2d_label(pc_scores_train[:, [dim_1,dim_2]], y_train)
# plt.legend(loc='center left', bbox_to_anchor=[1.01, 0.5], scatterpoints=3) # Add a legend outside the plot at specified point
# plt.xlabel('Dim {}'.format(dim_1))
# plt.ylabel('Dim {}'.format(dim_2))
# plt.show()


In [ ]:
# Classify_Function(pc_scores_train,y_train,pc_scores_val,y_val)

In [13]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X=enc.fit_transform(X_train_raw['CLASS'].values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["Class_"+str(int(i)) for i in range(X.shape[1])])
dfOneHot = pd.concat([dfOneHot, X_train_raw['N(0,1)']], axis=1)
X_train_SSS = pd.concat([X_train, dfOneHot], axis=1)


X=enc.fit_transform(X_val_raw['CLASS'].values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["Class_"+str(int(i)) for i in range(X.shape[1])])
dfOneHot = pd.concat([dfOneHot, X_val_raw['N(0,1)']], axis=1)
X_val_SSS = pd.concat([X_val, dfOneHot], axis=1)


X=enc.fit_transform(X_test_raw['CLASS'].values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["Class_"+str(int(i)) for i in range(X.shape[1])])
dfOneHot = pd.concat([dfOneHot, X_test_raw['N(0,1)']], axis=1)
X_test_SSS = pd.concat([X_test, dfOneHot], axis=1)


C:\Users\jsmyt\Anaconda3\envs\dme_project\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\jsmyt\Anaconda3\envs\dme_project\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "

In [ ]:
 sc2 = StandardScaler().fit(X_train_SSS.astype('float64'))


# Classify_Function(sc2.transform(X_train_SSS.astype('float64')),
#                   y_train,
#                   sc2.transform(X_val_SSS.astype('float64')),
#                   y_val)

In [ ]:
# #Classifying the test set
# sc3 = StandardScaler().fit(X_train.astype('float64'))
# #sc3.transform(X_train.astype('float64'))
# final_names = ["Random Forest", "Neural Net (Multi-layer perceptron)"]
# final_classifiers = [RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state),MLPClassifier(max_iter=1000, random_state=random_state)]

# ca_score = {} # Classification accuracy
# F1_scores = {} #F1 scores

# for name, clf in zip(final_names, final_classifiers):
#     clf.fit(sc3.transform(X_train.astype('float64')), y_train)
#     ca_score[name] = clf.score(sc3.transform(X_test.astype('float64')), y_test)
#     F1_scores[name] = f1_score(y_test,clf.predict(sc3.transform(X_test.astype('float64'))),average='macro')
    
# print('Classification performance on test set:')
# for clf in final_names:
#     print ("{}, accuracy: {:.3f}, f1-score: {:.3f}\n\n".format(clf, ca_score[clf], F1_scores[clf]))

In [32]:
sc2 = StandardScaler().fit(X_train_SSS.astype('float64'))
X_train_SSS_sc= sc2.transform(X_train_SSS.astype('float64'))
X_val_SSS_sc = sc2.transform(X_val_SSS.astype('float64'))
X_test_SSS_sc = sc2.transform(X_test_SSS.astype('float64'))

Backwards Elimination

In [ ]:
 %%time
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

# # # Build a classification task using 3 informative features
# # X, y = make_classification(n_samples=1000, n_features=25, n_informative=3,
# #                            n_redundant=2, n_repeated=0, n_classes=8,
# #                            n_clusters_per_class=1, random_state=0)

# # Create the RFE object and compute a cross-validated score.
# svc = SVC(kernel="linear")
# # The "accuracy" scoring is proportional to the number of correct
# # classifications
# rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
#               scoring='accuracy')
# rfecv.fit(X_train_SSS, y_train)

# print("Optimal number of features : %d" % rfecv.n_features_)

# # Plot number of features VS. cross-validation scores
# plt.figure()
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score (nb of correct classifications)")
# plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
# plt.show()


##############################################################

##Non SVC##
# # Build a classification task using 3 informative features
# X, y = make_classification(n_samples=1000, n_features=25, n_informative=3,
#                            n_redundant=2, n_repeated=0, n_classes=8,
#                            n_clusters_per_class=1, random_state=0)

# Create the RFE object and compute a cross-validated score.
rfc=SVC(kernel="linear", probability=True, random_state=random_state)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=rfc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X_train_SSS, y_train)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [27]:
type(X_train_SSS.columns[0])

str

In [26]:
ind_list=['AREA', 'IPEAK', 'COSMAG', 'ISKY', 'A_U', 'B_U', 'A_I', 'B_I', 'PRFMAG',
       'C_COSMAG', 'C_PRFMAG', 'GMAG_SDSS', 'RMAG_SDSS', 'IMAG_SDSS',
       'ELLIPTICITY', 'FILLING_FACTOR', 'Class_3']
type(ind_list[0])

str

In [39]:
#Get indices of most important variables
# ind_list=[]
# for i in range(len(X_train.columns)):
#     if rfecv.ranking_[i] == 1:
#         ind_list.append(i)
var_list=['AREA', 'IPEAK', 'COSMAG', 'ISKY', 'A_U', 'B_U', 'A_I', 'B_I', 'PRFMAG',
       'C_COSMAG', 'C_PRFMAG', 'GMAG_SDSS', 'RMAG_SDSS', 'IMAG_SDSS',
       'ELLIPTICITY', 'FILLING_FACTOR', 'Class_3']

ind_list=[]
for i in range(len(X_train_SSS.columns)):
    if X_train_SSS.columns[i] in var_list:
        ind_list.append(i)
        
bwe_data_train = X_train_SSS_sc[:, ind_list]
bwe_data_val=X_val_SSS_sc[:,ind_list]
bwe_data_test=X_test_SSS_sc[:,ind_list]

In [46]:
X_test_SSS.shape

(1565, 27)

In [37]:
bwe_data_train.shape

(10951, 17)

In [41]:
print('Order of colums originally: ', X_train.columns)
print('\n')
print('Indexes of variables selected: ',ind_list)
print('\n')
print('Actual variables selected: ',X_train_SSS.columns[ind_list])

Order of colums originally:  Index(['AREA', 'IPEAK', 'COSMAG', 'ISKY', 'A_U', 'B_U', 'THETA_U', 'A_I',
       'B_I', 'THETA_I', 'BLEND', 'QUALITY', 'PRFMAG', 'C_COSMAG', 'C_PRFMAG',
       'RA_SDSS', 'DEC_SDSS', 'GMAG_SDSS', 'RMAG_SDSS', 'IMAG_SDSS',
       'ELLIPTICITY', 'FILLING_FACTOR'],
      dtype='object')


Indexes of variables selected:  [0, 1, 2, 3, 4, 5, 7, 8, 12, 13, 14, 17, 18, 19, 20, 21, 25]


Actual variables selected:  Index(['AREA', 'IPEAK', 'COSMAG', 'ISKY', 'A_U', 'B_U', 'A_I', 'B_I', 'PRFMAG',
       'C_COSMAG', 'C_PRFMAG', 'GMAG_SDSS', 'RMAG_SDSS', 'IMAG_SDSS',
       'ELLIPTICITY', 'FILLING_FACTOR', 'Class_3'],
      dtype='object')


In [ ]:
bwe_data_train.shape

In [ ]:
y_train.shape

In [ ]:
bwe_data_val.shape

In [ ]:
y_val.shape

In [ ]:
# Classify_Function(sc2.transform(bwe_data_train.astype('float64')),
#                   y_train,
#                   sc2.transform(bwe_data_val.astype('float64')),
#                   y_val)

In [42]:

names = ["Logistic Regression", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net (Multi-layer perceptron)"]

classifiers = [
    LogisticRegression(),
#     SVC(kernel="linear", probability=True, random_state=random_state),
#     SVC(kernel='rbf', probability=True, random_state=random_state),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state),
    MLPClassifier(max_iter=1000, random_state=random_state)]

ca_score = {} # Classification accuracy
F1_scores = {} #F1 scores

for name, clf in zip(names, classifiers):
    clf.fit(bwe_data_train, y_train)
    ca_score[name] = clf.score(bwe_data_val, y_val)
    F1_scores[name] = f1_score(y_val,clf.predict(bwe_data_val),average='macro')
    
print('Classification performance on validation set:')
for clf in names:
    print ("{}, accuracy: {:.3f}, f1-score: {:.3f}".format(clf, ca_score[clf],F1_scores[clf]))

C:\Users\jsmyt\Anaconda3\envs\dme_project\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Classification performance on validation set:
Logistic Regression, accuracy: 0.870, f1-score: 0.862
Linear SVM, accuracy: 0.874, f1-score: 0.866
RBF SVM, accuracy: 0.881, f1-score: 0.875
Decision Tree, accuracy: 0.903, f1-score: 0.897


KeyError: 'Random Forest'